In [2]:
%run Utils/Utils.ipynb
%run Utils/Utils_Widgets.ipynb

Servo Motors Ready!
{'model': 24, 'firmware': (1, 29), 'hardware': 7, 'serialnumber': '9C9F9A86C0E09CC7A2E09DF70E4D3077'}
('Good', 0)
Lidar has started streaming data!
Object recognition model Ready!
shutdown request: [/ttb3_test] Reason: new node registered with same name


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-eefeaeb09446>", line 7, in readLidarThread
    for scan in lidar.iter_scans(scan_type='express', min_len = 100, max_buf_meas=False):
  File "/home/jetbot/.local/lib/python3.6/site-packages/rplidar.py", line 446, in iter_scans
    for new_scan, quality, angle, distance in iterator:
  File "/home/jetbot/.local/lib/python3.6/site-packages/rplidar.py", line 411, in iter_measures
    self._read_response(dsize))
  File "/home/jetbot/.local/lib/python3.6/site-packages/rplidar.py", line 466, in from_string
    raise ValueError('try to parse corrupted data ({})'.format(packet))
ValueError: try to parse corrupted data (bytearray(b'%\xccg%\xdf$\xccs$\x0f$\xcc\xb3#\x8f#\xbc{###\xbb\xc7"\x87"\xbb7"\xcf!\xbb\x83!G!\xbb\x0

In [ ]:
# %run Utils/Utils_DepthCam.ipynb
%run Utils/Utils_Camera.ipynb

In [ ]:
#!pip install paho.mqtt
%run Utils/Utils_MQTT.ipynb
stepCount = 1

# Parameters

In [ ]:
goal_x_coor = -1.6 #-1.95 #1.6  # in m, measured relative to the initial robot coordinates
goal_y_coor = 5.20 #5.5 #5.20  # in m, measured relative to the initial robot coordinates

running_control_thread = False

getObservation()
print(observation)

In [ ]:
# model_path = "Turtlebot3_0331.pth"
model_path = "Turtlebot3_LSTM.pth"
agent = torch.load(model_path)
agent.eval()
print('Model Loaded!')

In [ ]:
#getObservation()
memories = torch.ones((1,1,256))
masks = torch.ones((1,6))

# Initialize Logging

In [ ]:
import logging
import datetime

now = datetime.datetime.now()
dt_string = now.strftime("%Y%m%d__%H_%M_%S")

In [ ]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(filename="log_{}.txt".format(dt_string), level=logging.ERROR, format="%(asctime)s %(message)s")
logging.error("{},{},{},{}".format(real_x, real_y, real_angle, stepCount))

In [ ]:
# Test

# observation = [1]*41

# dists, value_out, memories = agent.get_dist_and_value(vec_inputs=[torch.Tensor(observation).cuda()], vis_inputs=[], masks=masks, memories=memories)
# actions = GetActionDiscrete(agent)  # LSTM
# # actions = GetAction(agent)          # MLP
# actions

# for dist in dists:
#     print(dist.entropy())
    
# for dist in dists:
#     print(dist.probs)
#     dist.probs=dist.probs*5
    
#     print(dist.probs)
    
# for dist in dists:
#     print(dist.probs)
    
# for dist in dists:
#     print(dist.entropy())
    
# actions = []
# for dist in dists:
#     actions.append(dist.sample().cpu().numpy()[0][0])
# print(actions)

# Control

In [ ]:
def ControlRobot(debug=False):
    
    global stepCount

    delay_counter = 0
    while True:

        # Get an Observation
        try:
            getObservation()
        except:
            continue

        updateStateWidgets()
        
        # Get Action
        #actions = GetAction(agent)         # Get act ion from MLP
        #actions = GetActionDiscrete(agent)  # Get action from LSTM
        
        actions = GetActionMod(agent)      # Get action from LSTM

        if delay_counter>5:
        
            # If still far away
            if observation[0] > 0.50:
                #print(f'Performing action = {actions}')
                act(actions)
                #a=0

            else:
                print("reached goal")
                break
                
        delay_counter+=1

            

In [ ]:
def showDepthcamFeed_TRT_CSI():
    global clockIsVisible, image
    global running_control_thread
    
    while True:

        # Obtain BBoxes + Labels from model
        detections = getDetections(image)
        
        # 85 = clock, 
        if 85 in detections : # or 31 in detections or 25 in detections or 85 in detections:
            clockIsVisible =1 
        else:
            clockIsVisible = 0
        
            
        # Run Control Thread
        if running_control_thread == False:
            
            controlThread = Thread(target = ControlRobot)
            controlThread.start()
            running_control_thread = True

In [ ]:
stepCount

In [ ]:
display(full_widget)

In [ ]:
# Delay Camera thread to make sure it works.
display(image_widget)

In [ ]:
# Start camera
execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
# showCSIFeed()
showDepthcamFeed_TRT_CSI()